## 2012-2021 Baltimore Victim-Based Crime exploratory data analysis

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st


In [2]:
# read in open baltimore 2012-2023 dataset
df_raw = pd.read_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data.csv")
df_raw.head()

C:\Users\johbr\AppData\Local\Temp\ipykernel_21948\1278201283.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data.csv")


,OBJECTID,CCNO,CrimeDateTime,CrimeCode,Location,Description,Inside_Outside,Weapon,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,Total_Incidents
0,1,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,NaN,PERSONAL_WEAPONS,833.0,F,15.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.273302,-76.692439,"(39.27330200992213,-76.69243902745305)",NaN,1
1,2,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,NaN,PERSONAL_WEAPONS,833.0,F,15.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.273302,-76.692439,"(39.27330200992213,-76.69243902745305)",NaN,1
2,3,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,NaN,PERSONAL_WEAPONS,833.0,F,27.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.273302,-76.692439,"(39.27330200992213,-76.69243902745305)",NaN,1
3,4,23F08231,2023/06/24 04:01:00+00,3JK,600 LUCIA AVE,ROBBERY,NaN,PERSONAL_WEAPONS,833.0,M,25.0,BLACK_OR_AFRICAN_AMERICAN,UNKNOWN,SOUTHWEST,YALE HEIGHTS,39.273302,-76.692439,"(39.27330200992213,-76.69243902745305)",NaN,1
4,5,23F08235,2023/06/24 03:45:00+00,5A,3200 LILY AVE,BURGLARY,NaN,NaN,922.0,M,48.0,NaN,HISPANIC_OR_LATINO,SOUTHERN,CHERRY HILL,39.246432,-76.636819,"(39.24643210111462,-76.63681903810716)",NaN,1


In [3]:
print(df_raw.iloc[:, 11])
"""
DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data.csv")

so, mixed values in 'race' column? doesn't seem right. is there a random float in there?
what's the data type error? if null, why not in "inside/outside"?...
...something weird is going on

Should be fixed by replacing csv read in:
  df_raw = pd.read_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data.csv", dtype={"Race": str})
But i wanna poke it first
"""

0         BLACK_OR_AFRICAN_AMERICAN
1         BLACK_OR_AFRICAN_AMERICAN
2         BLACK_OR_AFRICAN_AMERICAN
3         BLACK_OR_AFRICAN_AMERICAN
4                               NaN
                    ...            
568609    BLACK_OR_AFRICAN_AMERICAN
568610                      UNKNOWN
568611    BLACK_OR_AFRICAN_AMERICAN
568612    BLACK_OR_AFRICAN_AMERICAN
568613    BLACK_OR_AFRICAN_AMERICAN
Name: Race, Length: 568614, dtype: object


'\nDtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.\n  df_2023 = pd.read_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data.csv")\n\nso, mixed values in \'race\' column? doesn\'t seem right. is there a random float in there?\nwhat\'s the data type error? if null, why not in "inside/outside"?...\n...something weird is going on\n\nShould be fixed by replacing csv read in:\n  df_raw = pd.read_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data.csv", dtype={"Race": str})\nBut i wanna poke it first\n'

In [4]:
# clean 2023 data

# create copy of raw 2012-2023 data
df = df_raw

# split crime date and time
df[["CrimeDate", "CrimeTime"]] = df["CrimeDateTime"].str.split(" ", 1, expand=True)
# delete useless columns
del df['CCNO']
del df['OBJECTID']
del df["CrimeDateTime"]
df.rename(columns={'Inside_Outside': 'Inside/Outside'}, inplace=True)
# insert date and time into beginning of df
df.insert(0, "CrimeDate", df.pop("CrimeDate"))
df.insert(1, "CrimeTime", df.pop("CrimeTime"))

df.CrimeDate = pd.to_datetime(df.CrimeDate, errors='coerce')

df = df[df.CrimeDate <= '2021-04-30']
df.head()


C:\Users\johbr\AppData\Local\Temp\ipykernel_21948\2387077374.py:7: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[["CrimeDate", "CrimeTime"]] = df["CrimeDateTime"].str.split(" ", 1, expand=True)


,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,Total_Incidents
83615,2021-04-16,13:30:00+00,6E,1700 N BENTALOU ST,LARCENY,O,NaN,723.0,M,28.0,BLACK_OR_AFRICAN_AMERICAN,NaN,WESTERN,COPPIN HEIGHTS/ASH-CO-EAST,39.307800,-76.653923,"(39.3078,-76.653923)",STREET,1
83616,2021-04-16,10:32:00+00,4D,200 N EUTAW ST,AGG. ASSAULT,I,HANDS,114.0,F,50.0,BLACK_OR_AFRICAN_AMERICAN,NaN,CENTRAL,DOWNTOWN,39.291551,-76.621136,"(39.291551,-76.621136)",MARKET STALLS,1
83617,2021-04-16,15:20:00+00,6J,1000 PATAPSCO ST,LARCENY,NaN,NaN,942.0,F,25.0,WHITE,NaN,SOUTHERN,FEDERAL HILL,39.277654,-76.613765,"(39.277654,-76.613765)",NaN,1
83618,2021-04-16,07:00:00+00,7A,2200 WILKENS AVE,AUTO THEFT,O,NaN,834.0,F,20.0,UNKNOWN,NaN,SOUTHWEST,CARROLLTON RIDGE,39.280495,-76.650988,"(39.280495,-76.650988)",STREET,1
83619,2021-04-16,21:40:00+00,3BJ,1600 EDISON HWY,ROBBERY - CARJACKING,O,NaN,332.0,M,38.0,UNKNOWN,NaN,EASTERN,BEREA,39.309893,-76.573469,"(39.309892500982698,-76.573469434507444)",STREET,1
83620,2021-04-16,19:00:00+00,6J,1500 MONTPELIER ST,LARCENY,NaN,NaN,411.0,M,36.0,BLACK_OR_AFRICAN_AMERICAN,NaN,NORTHEAST,COLDSTREAM HOMESTEAD MONTEBELLO,39.321562,-76.598155,"(39.321562,-76.598155)",NaN,1
83621,2021-04-16,23:14:00+00,4B,5100 DARIEN RD,AGG. ASSAULT,NaN,KNIFE,441.0,M,53.0,UNKNOWN,NaN,NORTHEAST,CEDONIA,39.326021,-76.531202,"(39.326021,-76.531202)",NaN,1
83622,2021-04-16,19:00:00+00,4E,1900 MCCULLOH ST,COMMON ASSAULT,I,NaN,131.0,F,44.0,BLACK_OR_AFRICAN_AMERICAN,NaN,CENTRAL,DRUID HEIGHTS,39.307312,-76.633800,"(39.307312,-76.6338)",APT/CONDO - OCCUPIED,1
83623,2021-04-16,14:00:00+00,6D,EUTAW ST & N MARTIN LUTHER KING JR BL,LARCENY FROM AUTO,O,NaN,132.0,NaN,NaN,UNKNOWN,NaN,CENTRAL,MADISON PARK,39.300505,-76.621933,"(39.300505483305869,-76.621933243912167)",STREET,1
83624,2021-04-16,23:30:00+00,4E,200 ATHOLGATE LN,COMMON ASSAULT,I,NaN,823.0,NaN,36.0,UNKNOWN,NaN,SOUTHWEST,TEN HILLS,39.285527,-76.696817,"(39.285527,-76.696817)",APT/CONDO - OCCUPIED,1


In [5]:
df.to_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data_clean.csv")

In [6]:
df.describe()
# look at age - i'm surprised it's that high
# wonder how that breaks down by type of crime committed

# what does post mean?

,Post,Age,Latitude,Longitude,Total_Incidents
count,473711.000000,381852.000000,480542.000000,480542.000000,481408.0
mean,503.587244,37.926534,38.773137,-75.576983,1.0
std,261.031554,38.989416,4.549539,8.867924,0.0
min,111.000000,-7979.000000,0.000000,-76.711280,1.0
25%,234.000000,26.000000,39.287699,-76.648362,1.0
50%,511.000000,35.000000,39.302473,-76.613526,1.0
75%,724.000000,49.000000,39.326625,-76.585595,1.0
max,943.000000,8251.000000,39.372088,0.000000,1.0


## Weapon types

In [7]:
# weapon types
print(f"""The number of weapon types is {df.Weapon.nunique()}

{df.Weapon.unique()}

{df.Weapon.value_counts()}
""")

The number of weapon types is 10

[nan 'HANDS' 'KNIFE' 'FIREARM' 'OTHER' 'FIRE' 'KNIFE_CUTTING_INSTRUMENT'
 'PERSONAL_WEAPONS' 'BLUNT_OBJECT' 'HANDGUN' 'UNKNOWN']

FIREARM                     42435
OTHER                       28118
KNIFE                       16995
HANDS                        6865
FIRE                         2233
PERSONAL_WEAPONS               35
BLUNT_OBJECT                    2
KNIFE_CUTTING_INSTRUMENT        1
HANDGUN                         1
UNKNOWN                         1
Name: Weapon, dtype: int64



## Gender

In [8]:
# gender
gender_df = df.copy()

# move 'male' and 'female' into 'm' and 'f'
gender_df.Gender = df.Gender.replace({"Female": "F", "Male": "M"})

# drop all other genders
gender_df = gender_df[gender_df.Gender.isin({"M", "F", "U"})]

print(f"""The original data's value counts:
{df.Gender.value_counts()}

gender_df is:
{gender_df.Gender.value_counts()}
""")

gender_df.count()

The original data's value counts:
F             199493
M             181765
U              11528
Male            6722
Female           815
B                130
W                 20
N                 11
,                 10
Transgende         7
FB                 6
FW                 2
160                2
O                  2
120                1
A                  1
60                 1
8                  1
MB                 1
77                 1
17                 1
042819             1
165                1
S                  1
FM                 1
M\                 1
T                  1
FF                 1
FU                 1
50                 1
Name: Gender, dtype: int64

gender_df is:
F    200308
M    188487
U     11528
Name: Gender, dtype: int64



CrimeDate          400323
CrimeTime          400323
CrimeCode          400323
Location           398206
Description        400323
Inside/Outside     364482
Weapon              86191
Post               393685
Gender             400323
Age                369496
Race               400309
Ethnicity              81
District           393685
Neighborhood       393606
Latitude           399544
Longitude          399544
GeoLocation        400323
Premise            364146
Total_Incidents    400323
dtype: int64

## Inside / Outside

In [9]:
# indoor / outdoor
in_out_df = df.copy()
in_out_df['Inside/Outside'] = df['Inside/Outside'].replace({'Outside': 'O', 'Inside': 'I'})

in_out_df.dropna(subset=['Inside/Outside'], inplace=True)

print(f"""Original Indoor/Outdoor value counts:

{df['Inside/Outside'].value_counts()}


New Indoor/Outdoor value counts:
{in_out_df['Inside/Outside'].value_counts()}
""")


Original Indoor/Outdoor value counts:

I          215730
O          211354
Outside      6689
Inside        955
Name: Inside/Outside, dtype: int64


New Indoor/Outdoor value counts:
O    218043
I    216685
Name: Inside/Outside, dtype: int64

